In [1]:
from openai import AsyncAzureOpenAI
from neopipe.result import Ok, Result, Err, ExecutionResult
from neopipe.task import FunctionAsyncTask, ClassAsyncTask
from neopipe.async_pipeline import AsyncPipeline
from dotenv import load_dotenv, find_dotenv
import os
import logging
# import asyncio
import json
from neopipe.result import Ok, Err, Result
from typing import List
import re

In [2]:
logging.getLogger("openai").setLevel(logging.ERROR)
logging.getLogger("httpx").setLevel(logging.ERROR)
# logging.getLogger("neopipe").setLevel(logging.ERROR)

In [3]:
load_dotenv(find_dotenv("../.env"), override=True)

True

In [ ]:
class AsyncOpenAITask(ClassAsyncTask[List[dict], str]):
    """
    Sends messages to Azure OpenAI and returns the raw assistant reply.
    """
    def __init__(self, client: AsyncAzureOpenAI, model: str = "gpt-4o"):
        super().__init__()
        self.client = client
        self.model = model

    async def format_prompt(self, criterion: str) -> str:
        """Turn a single test criterion into a chat-messages list."""
        messages = [
            {
                "role": "system",
                "content": (
                    "You are an expert QA engineer. "
                    "Given a list of test criterion, generate a JSON test case. You always wrapt the response with a ````json ... ``` markdown fence. "
                    "You shuld have only following keys in the JSON: test_id, description, steps, expected_result. "
                    "The steps should be a list of strings, and the expected_result should be a string. "
                    "The test_id should be a unique identifier, like 'test_001'. "
                    "The description should be a short summary of the test case. "
                    "The steps should be a list of actions to perform in the test case. "
                    "The expected_result should be the expected outcome of the test case. "
                    "You should not include any other keys or values in the JSON. "
                    "You should not include any other text in the response, only the JSON. "
                    )
            },
            {
                "role": "user",
                "content": f"Create a JSON test case for: {criterion}"
            }
        ]
        return messages

    async def execute(self, res: Result[str, str]) -> Result[str, str]:
        if res.is_err():
            return res
        messages = res.unwrap()
        messages = await self.format_prompt(messages)
        try:
            completion = await self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                response_format={ "type": "json_object" },
            )
            # take the first choice
            text = completion.choices[0].message.content.strip()
            return Ok(text)
        except Exception as e:
            return Err(f"OpenAI error: {e}")

In [5]:
client = AsyncAzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
)

In [ ]:
# ───────────────────────────────────────────────────────────────
# 3) Run it on a batch of 5 criteria, in parallel
# ───────────────────────────────────────────────────────────────

async def main() -> ExecutionResult[List[Result[dict, str]], str]:
    criteria = [
        "User login fails when password is too short",
        "API returns 400 on invalid JSON body",
        "Form does not submit when required fields are blank",
        "Search feature returns relevant results",
        "Payment checkout times out after 30 seconds"
    ]
    # Wrap each criterion in an Ok(...)
    inputs = [Ok(c) for c in criteria]
    tasks = [AsyncOpenAITask(client, model="gpt-4o") for _ in range(len(inputs))]
    pipeline = AsyncPipeline.from_tasks(tasks=tasks)
    # debug=True to capture per-task Trace if you like
    exec_res = await pipeline.run(
        inputs,
        debug=True
    )

    if exec_res.is_err():
        print("Pipeline error:", exec_res.err())
        return

    # exec_res.result is List[Result[dict, str]]
    for idx, res in enumerate(exec_res.result, 1):
        if res.is_ok():
            print(f"\n📄 Test case {idx}:\n", json.dumps(res.unwrap(), indent=2))
        else:
            print(f"\n❌ Failed to generate test {idx}:", res.err())
    return exec_res.unwrap()

In [8]:
result_ai_call = await main()

2025-05-28 17:18:04 - neopipe.task - INFO - [AsyncOpenAITask] Attempt 1 - Task ID: 0ccf2aa8-e13f-482a-b8c6-0ed33e4577b8
2025-05-28 17:18:04 - neopipe.task - INFO - [AsyncOpenAITask] Attempt 1 - Task ID: 17425a41-2219-485f-94fa-dd0a3d495eb2
2025-05-28 17:18:04 - neopipe.task - INFO - [AsyncOpenAITask] Attempt 1 - Task ID: 229c01dd-a041-479c-a788-b3bc00cd5a78
2025-05-28 17:18:04 - neopipe.task - INFO - [AsyncOpenAITask] Attempt 1 - Task ID: 5f6a877f-6b13-498d-8b54-7d3e0204b21c
2025-05-28 17:18:04 - neopipe.task - INFO - [AsyncOpenAITask] Attempt 1 - Task ID: eacd3ac7-408f-4aec-82bb-f50bd48d0c06
2025-05-28 17:18:05 - neopipe.task - INFO - [AsyncOpenAITask] Success on attempt 1 - Task ID: 229c01dd-a041-479c-a788-b3bc00cd5a78
2025-05-28 17:18:05 - neopipe.task - INFO - [AsyncOpenAITask] Success on attempt 1 - Task ID: 5f6a877f-6b13-498d-8b54-7d3e0204b21c
2025-05-28 17:18:05 - neopipe.task - INFO - [AsyncOpenAITask] Success on attempt 1 - Task ID: eacd3ac7-408f-4aec-82bb-f50bd48d0c06
2025-05


📄 Test case 1:
 "{\n      \"test_id\": \"test_001\",\n      \"description\": \"User login fails when password is too short\",\n      \"steps\": [\n        \"Navigate to the login page\",\n        \"Enter a valid username\",\n        \"Enter a password with less than the minimum required characters\",\n        \"Click on the login button\"\n      ],\n      \"expected_result\": \"The login attempt fails and an error message indicating the password is too short is displayed\"\n    }"

📄 Test case 2:
 "{\n      \"test_id\": \"test_001\",\n      \"description\": \"API returns 400 on invalid JSON body\",\n      \"steps\": [\n        \"Send a POST request to the API endpoint with an invalid JSON body.\",\n        \"Ensure that the 'Content-Type' header is set to 'application/json'.\",\n        \"Observe the response status code and response body.\"\n      ],\n      \"expected_result\": \"API returns a 400 Bad Request status code and an error message indicating the JSON body is invalid.\"\n  

# Now Lets slap Another Pipelin on top of this Pipeline
The goal of this pipeline to take the output of the previous pipeline and apply a new transformation to it.


In [9]:
@FunctionAsyncTask.decorator()
async def parse_json(res: Result[str, str]) -> Result[dict, str]:
    """
    Parse out a JSON object from a possibly fenced text block and return it.

    This will look for a Markdown-style fence:
    ```json
    { ... }
    ```
    and extract the `{ ... }` portion before loading. Falls back to
    trying to load the entire string if no fence is found.

    Args:
        res: A Result wrapping the raw assistant reply (string).

    Returns:
        Ok(dict): on successful JSON parsing.
        Err(str): on parse failure or if input was Err.
    """
    if res.is_err():
        return res

    raw = res.unwrap()
    try:
        data = json.loads(raw)
        return Ok(data)


    except Exception as e:
        logging.info(f"JSON parse error: {e} in text: {raw}")
        return Err(f"JSON parse error: {e}")

In [10]:
# basic test
async def test_parse_json(input: Result[str, str]) -> None:
    test_input = input
    
    result = await parse_json(test_input)
    
    if result.is_ok():
        print("Parsed JSON:", result.unwrap())
    else:
        print("Error parsing JSON:", result.err())

# Run the test
await test_parse_json(result_ai_call[0])

2025-05-28 17:18:28 - neopipe.task - INFO - [parse_json] Attempt 1 - Task ID: affaec01-5c26-402b-a124-966902f715bb
2025-05-28 17:18:28 - neopipe.task - INFO - [parse_json] Success on attempt 1 - Task ID: affaec01-5c26-402b-a124-966902f715bb


Parsed JSON: {'test_id': 'test_001', 'description': 'User login fails when password is too short', 'steps': ['Navigate to the login page', 'Enter a valid username', 'Enter a password with less than the minimum required characters', 'Click on the login button'], 'expected_result': 'The login attempt fails and an error message indicating the password is too short is displayed'}


In [40]:
await test_parse_json(result_ai_call[3])

2025-05-28 16:31:21 - neopipe.task - INFO - [parse_json] Attempt 1 - Task ID: f2718879-d29b-4952-96f2-7bacbf3081d3
2025-05-28 16:31:21 - neopipe.task - INFO - [parse_json] Success on attempt 1


Parsed JSON: {'test_id': 'test_001', 'description': 'Search feature returns relevant results', 'steps': ['Navigate to the search bar', 'Enter a search query', 'Click the search button', 'Review the search results'], 'expected_result': 'Search results are relevant to the search query entered'}


In [11]:
# Pipeline to parse the AI-generated JSON
parse_json_task = [parse_json for _ in range(len(result_ai_call))]
pipeline_parse = AsyncPipeline.from_tasks(tasks=parse_json_task)
# Run the parsing pipeline
exec_parse_res = await pipeline_parse.run(
    result_ai_call,
    debug=False
)
if exec_parse_res.is_err():
    print("Parsing pipeline has error error.")
# Print the parsed results
for idx, res in enumerate(exec_parse_res.result, 1):
    if res.is_ok():
        print(f"\n📄 Parsed Test case {idx}:\n", res.unwrap())
    else:
        print(f"\n❌ Failed to parse test {idx}:", res.err())

2025-05-28 17:18:36 - neopipe.task - INFO - [parse_json] Attempt 1 - Task ID: affaec01-5c26-402b-a124-966902f715bb
2025-05-28 17:18:36 - neopipe.task - INFO - [parse_json] Success on attempt 1 - Task ID: affaec01-5c26-402b-a124-966902f715bb
2025-05-28 17:18:36 - neopipe.task - INFO - [parse_json] Attempt 1 - Task ID: affaec01-5c26-402b-a124-966902f715bb
2025-05-28 17:18:36 - neopipe.task - INFO - [parse_json] Success on attempt 1 - Task ID: affaec01-5c26-402b-a124-966902f715bb
2025-05-28 17:18:36 - neopipe.task - INFO - [parse_json] Attempt 1 - Task ID: affaec01-5c26-402b-a124-966902f715bb
2025-05-28 17:18:36 - neopipe.task - INFO - [parse_json] Success on attempt 1 - Task ID: affaec01-5c26-402b-a124-966902f715bb
2025-05-28 17:18:36 - neopipe.task - INFO - [parse_json] Attempt 1 - Task ID: affaec01-5c26-402b-a124-966902f715bb
2025-05-28 17:18:36 - neopipe.task - INFO - [parse_json] Success on attempt 1 - Task ID: affaec01-5c26-402b-a124-966902f715bb
2025-05-28 17:18:36 - neopipe.task -


📄 Parsed Test case 1:
 {'test_id': 'test_001', 'description': 'User login fails when password is too short', 'steps': ['Navigate to the login page', 'Enter a valid username', 'Enter a password with less than the minimum required characters', 'Click on the login button'], 'expected_result': 'The login attempt fails and an error message indicating the password is too short is displayed'}

📄 Parsed Test case 2:
 {'test_id': 'test_001', 'description': 'API returns 400 on invalid JSON body', 'steps': ['Send a POST request to the API endpoint with an invalid JSON body.', "Ensure that the 'Content-Type' header is set to 'application/json'.", 'Observe the response status code and response body.'], 'expected_result': 'API returns a 400 Bad Request status code and an error message indicating the JSON body is invalid.'}

📄 Parsed Test case 3:
 {'test_id': 'test_001', 'description': 'Form does not submit when required fields are blank', 'steps': ['Navigate to the form page', 'Leave all required f

In [22]:
print(exec_parse_res)

ExecutionResult(result=[Ok({'test_id': 'test_001', 'description': 'User login fails when password is too short', 'steps': ['Navigate to the login page', 'Enter a valid username', 'Enter a password with less than the minimum required characters', 'Click on the login button'], 'expected_result': 'The login attempt fails and an error message indicating the password is too short is displayed'}), Ok({'test_id': 'test_001', 'description': 'API returns 400 on invalid JSON body', 'steps': ['Send a POST request to the API endpoint with an invalid JSON body.', "Ensure that the 'Content-Type' header is set to 'application/json'.", 'Observe the response status code and response body.'], 'expected_result': 'API returns a 400 Bad Request status code and an error message indicating the JSON body is invalid.'}), Ok({'test_id': 'test_001', 'description': 'Form does not submit when required fields are blank', 'steps': ['Navigate to the form page', 'Leave all required fields blank', 'Attempt to submit th

# Combined Async Task in Sequence and then Run each In parallel
Now lets combine to task in sequence to create a sequecial pipeline and then run each task in parallel.

In [12]:
criteria = [
        "User login fails when password is too short",
        "API returns 400 on invalid JSON body",
        "Form does not submit when required fields are blank",
        "Search feature returns relevant results",
        "Payment checkout times out after 30 seconds"
    ]

In [13]:
seq_inputs = [Ok(c) for c in criteria]

In [ ]:
pipeline_unit = AsyncPipeline.from_tasks([
    AsyncOpenAITask(client, model="gpt-4o")],
    parse_json
)

In [ ]:
multi_task_pipelines = [pipeline_unit for _ in range(len(seq_inputs))]

In [26]:
parallel_result = await AsyncPipeline.run_parallel(
    pipelines=multi_task_pipelines,
    inputs=seq_inputs,
    debug=False
)

2025-05-29 17:42:40 - neopipe.task - INFO - [AsyncOpenAITask] Attempt 1 - Task ID: ba9c1cd5-834e-48b2-8878-d959cf54df3c
2025-05-29 17:42:40 - neopipe.task - INFO - [AsyncOpenAITask] Attempt 1 - Task ID: d801faa4-fa01-45b8-8bdf-1154ffc99a31
2025-05-29 17:42:40 - neopipe.task - INFO - [AsyncOpenAITask] Attempt 1 - Task ID: 62279e60-e9fc-49b4-9134-389a3813377e
2025-05-29 17:42:40 - neopipe.task - INFO - [AsyncOpenAITask] Attempt 1 - Task ID: 9c2eff29-3aad-4c07-9cac-c5e11c3e8a77
2025-05-29 17:42:40 - neopipe.task - INFO - [AsyncOpenAITask] Attempt 1 - Task ID: b1b744f2-5162-4c3c-92bd-dd9f5ea352f8
2025-05-29 17:42:41 - neopipe.task - INFO - [AsyncOpenAITask] Success on attempt 1 - Task ID: 62279e60-e9fc-49b4-9134-389a3813377e
2025-05-29 17:42:41 - neopipe.task - INFO - [AsyncOpenAITask] Success on attempt 1 - Task ID: 9c2eff29-3aad-4c07-9cac-c5e11c3e8a77
2025-05-29 17:42:41 - neopipe.task - INFO - [AsyncOpenAITask] Success on attempt 1 - Task ID: d801faa4-fa01-45b8-8bdf-1154ffc99a31
2025-05

In [27]:
parallel_result.unwrap()

[Ok('{\n      "test_id": "test_001",\n      "description": "User login fails when password is too short",\n      "steps": [\n        "Navigate to the login page",\n        "Enter a valid username in the username field",\n        "Enter a password shorter than the required minimum length in the password field",\n        "Click the \'Login\' button"\n      ],\n      "expected_result": "The login attempt should fail and the user should see an error message indicating that the password is too short."\n    }'),
 Ok('{\n      "test_id": "test_001",\n      "description": "API returns 400 on invalid JSON body",\n      "steps": [\n        "Send a POST request to the API endpoint with an invalid JSON body",\n        "Observe the response status code"\n      ],\n      "expected_result": "The API should return a 400 status code indicating a bad request"\n    }'),
 Ok('{\n        "test_id": "test_001",\n        "description": "Form does not submit when required fields are blank",\n        "steps": 

# Some Issues
If we create one pipeline and run it in parallel, it produce single id for all tasks so we should create task/pipeline on the fly or create a func that create pipeline on the fly.

In [ ]:
def get_unit_pipeline():
    pipeline_unit = AsyncPipeline.from_tasks([
        AsyncOpenAITask(client, model="gpt-4o")],
        parse_json
        )
    return pipeline_unit

In [ ]:
input_tests = [[{}, {}, {}], [{}, {}, {}], [{}, {}, {}], [{}, {}, {}], [{}, {}, {}]]

In [ ]:
class AsyncGenerateTestCasesTask(ClassAsyncTask[List[dict], str]):
    """
    Sends messages to Azure OpenAI and returns the raw assistant reply.
    """
    def __init__(self, client: AsyncAzureOpenAI, model: str = "gpt-4o", system_prompt: str = None, user_prompt: str = None):
        super().__init__()
        self.client = client
        self.model = model
        self.system_prompt = system_prompt
        self.user_prompt = user_prompt
    
    async def check_correctness(self) -> bool:
        ...


    async def format_llm_request(self, criterion: list[dict]) -> str:
        """Turn a single test criterion into a chat-messages list."""
        
        ...

    async def execute(self, res: Result[list[dict], str]) -> Result[list[dict], str]:
        ...
        # Step 1 create Context for the LLM: format_llm_request(res)
        # Step 2: with some retry in a 4 loop call the 
            # response = (api call)
            # check_correctness(response) 3 send 3 came back then break the retry
            # Else try again

In [ ]:
@FunctionAsyncTask.decorator()
async def parse_json(res: Result[list[dict], str]) -> Result[list[dict], str]:
    """
    Parse out a JSON object from a possibly fenced text block and return it.

    This will look for a Markdown-style fence:
    ```json
    { ... }
    ```
    and extract the `{ ... }` portion before loading. Falls back to
    trying to load the entire string if no fence is found.

    Args:
        res: A Result wrapping the raw assistant reply (string).

    Returns:
        Ok(dict): on successful JSON parsing.
        Err(str): on parse failure or if input was Err.
    """
    ...

In [ ]:
input_tests = [Ok(c) for c in input_tests]

In [ ]:
def get_unit_pipeline():
    pipeline_unit = AsyncPipeline.from_tasks([
        AsyncOpenAITask(client, model="gpt-4o")],
        parse_json
        )
    return pipeline_unit

In [ ]:
multi_task_pipelines = [pipeline_unit for _ in range(len(input_tests))]

In [ ]:
parallel_result = await AsyncPipeline.run_parallel(
    pipelines=multi_task_pipelines,
    inputs=input_tests,
    debug=False
)